# SSM 계산 : 차량 페어, 롱타입
* Heatmap으로 나타내기
* 시공도로 나타내기

# Import

In [37]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
import os

import math # arctangent; math.atan 사용 목적
import statistics

from SSM_base import *
from SSM_time_based import *
from SSM_deceleration_based import *
from SSM_energy_based import *

import pyarrow.parquet as pq
#df = pq.read_pandas('data.parquet').to_pandas()

In [38]:
#warnings.filterwarnings('ignore')

# Load Dataset

In [39]:
working_dir = 'D:/OneDrive/Projects/2023_SSM_Feasibility/Dataset'

In [40]:
folder_name = '02_processed_long'
file_name = 'merging_type1_03frm_0.10sec.parquet'
file_path = os.path.join(working_dir, folder_name, file_name)

In [41]:
df = pq.read_pandas(file_path).to_pandas()

df.rename({'LV_V_len' : 'LV_len',
           'LV_V_wid' : 'LV_wid'}, axis = 1, inplace = True)

In [42]:
df.head(3)

,veh_id,frm,local_x,local_y,V_len,V_wid,velocity,acc,lane,Time,...,LV_Lane_00,LV_Lane_99,LV_Lane_change,LV_Lane_leave,LV_Lane_change_direction,LV_LC_CF,D_x,D_y,D,D_gap
0,2,438,37.28125,7.726562,4.902344,2.064453,92.1875,2.812500,U2,14.6,...,U3,U3,None,None,Straight,CF,93.59375,3.273438,93.650978,NaN
1,2,441,39.81250,7.785156,4.902344,2.064453,91.7500,-1.228516,U2,14.7,...,U3,U3,None,None,Straight,CF,93.31250,3.246094,93.368942,-0.282036
2,2,444,42.40625,7.882812,4.902344,2.064453,92.4375,1.814453,U2,14.8,...,U3,U3,None,None,Straight,CF,93.09375,3.187500,93.148300,-0.220642


In [43]:
list(df.columns)

['veh_id',
 'frm',
 'local_x',
 'local_y',
 'V_len',
 'V_wid',
 'velocity',
 'acc',
 'lane',
 'Time',
 'local_x_before',
 'local_y_before',
 'delta_local_x',
 'delta_local_y',
 'velocity_x',
 'velocity_y',
 'velocity_x_before',
 'velocity_y_before',
 'delta_velocity_x',
 'delta_velocity_y',
 'acc_x',
 'acc_y',
 'lane_past',
 'Lane_record',
 'Lane_record_split',
 'Lane_00',
 'Lane_99',
 'Lane_change',
 'Lane_leave',
 'Lane_change_direction',
 'LC_CF',
 'LV_ID',
 'LV_type',
 'LV_local_x',
 'LV_local_y',
 'LV_len',
 'LV_wid',
 'LV_velocity',
 'LV_acc',
 'LV_lane',
 'LV_Time',
 'LV_local_x_before',
 'LV_local_y_before',
 'LV_delta_local_x',
 'LV_delta_local_y',
 'LV_velocity_x',
 'LV_velocity_y',
 'LV_velocity_x_before',
 'LV_velocity_y_before',
 'LV_delta_velocity_x',
 'LV_delta_velocity_y',
 'LV_acc_x',
 'LV_acc_y',
 'LV_lane_past',
 'LV_Lane_record',
 'LV_Lane_record_split',
 'LV_Lane_00',
 'LV_Lane_99',
 'LV_Lane_change',
 'LV_Lane_leave',
 'LV_Lane_change_direction',
 'LV_LC_CF',
 'D_

# Calculation

## Settings
* 공통 상수 설정

In [44]:
threshold_TTC = 1.5 # time-based SSM 임계값
threshold_DRAC = 3.4

tmsec = 0.1 # 각 row별 시간단위(초)
g = 9.81 # 중력가속도
f = 0.35 # 마찰계수. friction coeefficient
delta_t = 1 # Reaction Delay time. 운전자 반응시간
MADR = 8.45 # 차량의 최대 감속 능력. 날씨에 따라, 차종에 따라 다르다. Saccomanno et al.(2008) 참조
deceleration_rate = 0.3 * g # 차량 감속도. 3.3도 됨

## Time-based SSM
* Temporal Proximal Indicator

In [45]:
tqdm.pandas()

In [46]:
# 꼭지점 4개 만들기
# 각 차량의 1,2,3,4번 꼭지점 좌표를 구한다.
df['V_point_1'], df['V_point_2'], df['V_point_3'], df['V_point_4'] = zip(*df.progress_apply(lambda x: points(x.local_x, x.local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
df['LV_point_1'], df['LV_point_2'], df['LV_point_3'], df['LV_point_4'] = zip(*df.progress_apply(lambda x: points(x.LV_local_x, x.LV_local_y, x.LV_velocity_x, x.LV_velocity_y, x.LV_wid, x.LV_len), axis = 1))

100%|███████████████████████████████████████████████████████████████████████| 265505/265505 [00:12<00:00, 21363.87it/s]


In [47]:
## Nearest_point 추출
df['V_LV_point_num'], df['V_LV_point_n'], df['LV_point_n'], df['V_LV_point_num2'], df['V_LV_point_n2'], df['LV_point_n2'] = zip(*df.progress_apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

100%|███████████████████████████████████████████████████████████████████████| 265505/265505 [00:18<00:00, 14164.56it/s]


In [48]:
## Shortest Distance 구하기 : ACT 용
df['V_LV_point_delta'], df['V_LV_shortest_vector'], df['V_LV_shortest_distance'] = zip(*df.progress_apply(lambda x: shortest_distance(
    x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

100%|███████████████████████████████████████████████████████████████████████| 265505/265505 [00:20<00:00, 12761.43it/s]


In [49]:
debug = []

#df['ip1'], df['ip2'], df['cos_theta'], df['theta'], df['degX'],
df['potential_conflict_type'], df['degX'] = zip(*df.progress_apply(lambda x: potential_conflict_type(x.LV_type, x.D_gap, x.V_LV_point_num, x.V_LV_point_num2, x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y, x.local_x, x.local_y, x.LV_local_x, x.LV_local_y, x.V_len, x.LV_len), axis = 1))

100%|███████████████████████████████████████████████████████████████████████| 265505/265505 [00:24<00:00, 11039.03it/s]


In [50]:
# 2차원 SSM
df['T2'] = df.progress_apply(lambda x: T2(
    x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
    x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1)

100%|████████████████████████████████████████████████████████████████████████| 265505/265505 [00:39<00:00, 6663.96it/s]


In [51]:
#df[['ip1', 'ip2', 'cos_theta', 'theta', 'degX', 'potential_conflict_type']]

In [52]:
df['TTC'] = df.progress_apply(lambda x: TTC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
#df['ITTC'] = df.progress_apply(lambda x: ITTC(x.potential_conflict_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
df['MTTC'] = df.progress_apply(lambda x: MTTC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, x.V_len, x.LV_len, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
#df['MTTC_conflict_occurs'] = df.progress_apply(lambda x: conflict_occurs(x.potential_conflict_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
#df['CI_MTTC'] = df.progress_apply(lambda x: CI_MTTC(x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, x.MTTC), axis = 1)

#df['TIT'] = df.progress_apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
#df['TET'] = df.progress_apply(lambda x: TET(x.TTC, threshold_TTC, tmsec), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 265505/265505 [00:25<00:00, 10259.78it/s]


In [53]:
df['ACT'], df['total_rel'], df['d_delta'] = zip(*df.progress_apply(lambda x: ACT(
    x.velocity_x, x.velocity_y, x.velocity_x_before, x.velocity_y_before,
    x.LV_velocity_x, x.LV_velocity_y, x.LV_velocity_x_before, x.LV_velocity_y_before,
    x.acc_x, x.acc_y, x.LV_acc_x, x.LV_acc_y, 
    x.V_LV_shortest_vector, x.V_LV_shortest_distance), axis = 1))

100%|███████████████████████████████████████████████████████████████████████| 265505/265505 [00:26<00:00, 10081.15it/s]


In [54]:
df['pPET'], df['time_gap'] = zip(*df.progress_apply(lambda x: pPET(
    x.potential_conflict_type, x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
    x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
    x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
    x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1))

100%|████████████████████████████████████████████████████████████████████████| 265505/265505 [00:55<00:00, 4783.34it/s]


In [55]:
#df['TIT2'] = df.progress_apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
#df['TIACT'] = df.progress_apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)

In [56]:
# df['SSCR_TTC_prime'] = df.progress_apply(lambda x: SSCR_TTC_prime(
#     x.potential_conflict_type, x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
#     x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
#     x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4,
#     x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y), axis = 1)

In [57]:
# df['SSCR_R'] = df.progress_apply(lambda x: SSCR_R(x.TTC), axis = 1)
# df['SSCR_Y'] = df.progress_apply(lambda x: SSCR_Y(x.TTC, x.pPET, x.SSCR_TTC_prime), axis = 1)
# df['SSCR'] = df['SSCR_R'] + df['SSCR_Y']

# # df = TA(df) # Time to Accident 계산
# # df['critical_speed'] = df.apply(lambda x: critical_speed(g, f, x.pPET), axis = 1)
# # df['TA_CS'] = df.apply(lambda x: TA_CS(x.TA, x.critical_speed), axis = 1) # TA/CS 계산

## Deceleration-based SSM

In [58]:
df['PSD'] = df.progress_apply(lambda x: PSD(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, f, g, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['DSS'] = df.progress_apply(lambda x: DSS(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.V_len, x.LV_len, g, f, delta_t, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
#df['TIDSS'] = df.progress_apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)

df['PICUD'] = df.progress_apply(lambda x: PICUD(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.D_x, delta_t, deceleration_rate, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['DRAC'] = df.progress_apply(lambda x: DRAC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.D_x, x.velocity_x, x.LV_velocity_x, x.LV_len, x.V_len, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
#df['TIDRAC'] = df.apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)

df['MDRAC'] = df.progress_apply(lambda x: MDRAC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.TTC, delta_t, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['DCIA'] = df.progress_apply(lambda x: DCIA(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, delta_t, x.D_x, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['unsafety'] = df.progress_apply(lambda x: unsafety(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, deceleration_rate, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['MTC'] = df.progress_apply(lambda x: MTC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.D_x, x.V_len, x.LV_len, x.velocity_x, x.LV_velocity, delta_t, deceleration_rate, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
df['MMTC'] = df.progress_apply(lambda x: MMTC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, g, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

df['TIT'] = df.progress_apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
df['TIT2'] = df.progress_apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
df['TGIT'] = df.progress_apply(lambda x: TIT(x.time_gap, threshold_TTC, tmsec), axis = 1)
df['TIACT'] = df.progress_apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)

df['TIDSS'] = df.progress_apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)
df['TIDRAC'] = df.progress_apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)
df['TIMDRAC'] = df.progress_apply(lambda x: TIDRAC(x.MDRAC, threshold_DRAC), axis = 1)


#df['RCRI'] = df.progress_apply(lambda x: RCRI(x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, x.acc_x, x.LV_acc_x, x.D_x, delta_t, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)
#df['TERCRI'] = df.progress_apply(lambda x: TERCRI(x.RCRI, tmsec), axis = 1)

100%|███████████████████████████████████████████████████████████████████████| 265505/265505 [00:04<00:00, 64827.89it/s]


## Energy-based SSM

In [59]:
# df['CIF'] = df.progress_apply(lambda x: CIF(x.velocity_x, x.TTC), axis = 1)
# df['CIF2'] = df.progress_apply(lambda x: CIF(x.velocity, x.T2), axis = 1)
# df['ACIF'] = df.progress_apply(lambda x: CIF(x.velocity, x.ACT), axis = 1)

# Save

In [60]:
delete_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4', 
              'LV_point_1', 'LV_point_2', 'LV_point_3', 'LV_point_4',
              'V_LV_point_num', 'V_LV_point_n', 'LV_point_n', 'V_LV_point_num2',
              'V_LV_point_n2',	'LV_point_n2',	'V_LV_point_delta',	'V_LV_shortest_vector',
              'V_LV_shortest_distance', 'total_rel', 'd_delta']

df2 = df.copy()
df2.drop(delete_cols, inplace = True, axis = 1)

df2 = df2.dropna(subset = ['veh_id', 'frm'])

In [61]:
folder_name = '03_SSM'
save_name = file_name
save_path = os.path.join(working_dir, folder_name, save_name)

df2.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 

In [62]:
df2[['pPET', 'T2', 'ACT']]

,pPET,T2,ACT
0,NaN,NaN,NaN
1,20.362998,NaN,NaN
2,14.790288,NaN,15.554467
3,15.199430,NaN,NaN
4,23.551986,NaN,NaN
...,...,...,...
265500,NaN,NaN,NaN
265501,NaN,NaN,NaN
265502,NaN,NaN,NaN
265503,NaN,NaN,NaN


In [63]:
# tqdm.pandas()

# folder_name = '03_SSM'
# file_name = 'merging_type1_03frm_0.10sec.parquet'
# file_path = os.path.join(working_dir, folder_name, file_name)

# df = pq.read_pandas(file_path).to_pandas()

# # df['MMTC'] = df.progress_apply(lambda x: MMTC(x.potential_conflict_type, x.LV_type, x.local_x, x.LV_local_x, x.velocity_x, x.LV_velocity_x, g, x.V_wid, x.LV_wid, x.local_y, x.LV_local_y, x.velocity_y, x.LV_velocity_y), axis = 1)

# # 꼭지점 4개 만들기
# # 각 차량의 1,2,3,4번 꼭지점 좌표를 구한다.
# df['V_point_1'], df['V_point_2'], df['V_point_3'], df['V_point_4'] = zip(*df.progress_apply(lambda x: points(x.local_x, x.local_y, x.velocity_x, x.velocity_y, x.V_wid, x.V_len), axis = 1))
# df['LV_point_1'], df['LV_point_2'], df['LV_point_3'], df['LV_point_4'] = zip(*df.progress_apply(lambda x: points(x.LV_local_x, x.LV_local_y, x.LV_velocity_x, x.LV_velocity_y, x.LV_wid, x.LV_len), axis = 1))

# ## Nearest_point 추출
# df['V_LV_point_num'], df['V_LV_point_n'], df['LV_point_n'], df['V_LV_point_num2'], df['V_LV_point_n2'], df['LV_point_n2'] = zip(*df.progress_apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

# ## Shortest Distance 구하기 : ACT 용
# df['V_LV_point_delta'], df['V_LV_shortest_vector'], df['V_LV_shortest_distance'] = zip(*df.progress_apply(lambda x: shortest_distance(
#     x.V_LV_point_num, x.V_LV_point_n, x.LV_point_n, 
#     x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, 
#     x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

# ## Nearest_point 추출
# df['V_LV_point_num'], df['V_LV_point_n'], df['LV_point_n'], df['V_LV_point_num2'], df['V_LV_point_n2'], df['LV_point_n2'] = zip(*df.progress_apply(lambda x: nearest_point(x.V_point_1, x.V_point_2, x.V_point_3, x.V_point_4, x.LV_point_1, x.LV_point_2, x.LV_point_3, x.LV_point_4), axis = 1))

# debug = []

# #df['ip1'], df['ip2'], df['cos_theta'], df['theta'], df['degX'],
# df['potential_conflict_type'], df['degX'] = zip(*df.progress_apply(lambda x: potential_conflict_type(x.LV_type, x.D_gap, x.V_LV_point_num, x.V_LV_point_num2, x.velocity_x, x.velocity_y, x.LV_velocity_x, x.LV_velocity_y, x.local_x, x.local_y, x.LV_local_x, x.LV_local_y, x.V_len, x.LV_len), axis = 1))

# df['ACT'], df['total_rel'], df['d_delta'] = zip(*df.progress_apply(lambda x: ACT(
#     x.velocity_x, x.velocity_y, x.velocity_x_before, x.velocity_y_before,
#     x.LV_velocity_x, x.LV_velocity_y, x.LV_velocity_x_before, x.LV_velocity_y_before,
#     x.acc_x, x.acc_y, x.LV_acc_x, x.LV_acc_y, 
#     x.V_LV_shortest_vector, x.V_LV_shortest_distance), axis = 1))

# delete_cols = ['V_point_1', 'V_point_2', 'V_point_3', 'V_point_4', 
#               'LV_point_1', 'LV_point_2', 'LV_point_3', 'LV_point_4',
#               'V_LV_point_num', 'V_LV_point_n', 'LV_point_n', 'V_LV_point_num2',
#               'V_LV_point_n2',	'LV_point_n2',	'V_LV_point_delta',	'V_LV_shortest_vector',
#               'V_LV_shortest_distance', 'total_rel', 'd_delta']
#               #'V_rel', 'acc_rel', 'angle_rel']

# df2 = df.copy()
# df2.drop(delete_cols, inplace = True, axis = 1)


# folder_name = '03_SSM'
# save_name = file_name
# save_path = os.path.join(working_dir, folder_name, save_name)

# df2.to_parquet(save_path, engine = 'fastparquet', compression = 'gzip') 

In [64]:
df[['V_LV_shortest_distance', 'total_rel', 'd_delta', 'T2', 'TTC', 'MTTC', 'ACT']]

,V_LV_shortest_distance,total_rel,d_delta,T2,TTC,MTTC,ACT
0,NaN,None,NaN,NaN,NaN,NaN,NaN
1,91.207920,None,NaN,NaN,NaN,NaN,NaN
2,90.971394,"[4.23048962175395, 4.038407410127604]",5.84857,NaN,NaN,NaN,15.554467
3,90.711067,"[-2.248533892037565, -2.414158409469214]",NaN,NaN,NaN,NaN,NaN
4,90.482678,"[-2.86151369616484, -3.0681767122933317]",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
265500,NaN,None,NaN,NaN,NaN,NaN,NaN
265501,NaN,None,NaN,NaN,NaN,NaN,NaN
265502,NaN,None,NaN,NaN,NaN,NaN,NaN
265503,NaN,None,NaN,NaN,NaN,NaN,NaN


In [65]:
# df['TIT'] = df.progress_apply(lambda x: TIT(x.TTC, threshold_TTC, tmsec), axis = 1)
# df['TIT2'] = df.progress_apply(lambda x: TIT(x.T2, threshold_TTC, tmsec), axis = 1)
# df['TGIT'] = df.progress_apply(lambda x: TIT(x.time_gap, threshold_TTC, tmsec), axis = 1)
# df['TIACT'] = df.progress_apply(lambda x: TIT(x.ACT, threshold_TTC, tmsec), axis = 1)

# df['TIDSS'] = df.progress_apply(lambda x: TIDSS(x.DSS, tmsec), axis = 1)
# df['TIDRAC'] = df.progress_apply(lambda x: TIDRAC(x.DRAC, threshold_DRAC), axis = 1)
# df['TIMDRAC'] = df.progress_apply(lambda x: TIDRAC(x.MDRAC, threshold_DRAC), axis = 1)